# 实作情绪分析(Sentiment Analysis)

## 载入IMDB资料集 

In [25]:
import torch
from torchtext.datasets import IMDB

imdb = IMDB(split='train')

type(imdb)

C:\anaconda3\lib\site-packages\torch\utils\data\datapipes\utils\common.py:24: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn(
C:\anaconda3\lib\site-packages\torch\utils\data\datapipes\iter\selecting.py:54: UserWarning: Lambda function is not supported for pickle, please use regular python function or functools.partial instead.
  warnings.warn("Lambda function is not supported for pickle, please use "


torch.utils.data.datapipes.iter.callable.MapperIterDataPipe

## 取得下一笔资料测试

In [90]:
train_iter = iter(IMDB(split='train'))

data = next(train_iter)
data

('neg',
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

## 判断GPU是否存在

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## 词汇表处理

In [7]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# 分词
tokenizer = get_tokenizer('basic_english')

# 建立 Generator 函数
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

# 由 train_iter 建立词汇字典
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

# 设定预设的索引值
vocab.set_default_index(vocab["<unk>"])

In [8]:
# 测试词汇字典，取得单字的索引值
vocab(['here', 'is', 'an', 'example'])

[131, 9, 40, 464]

## 参数设定

In [91]:
EPOCHS = 10 # 训练周期数
LR = 5  # 学习率
BATCH_SIZE = 64 # 训练批量
# 取得标注个数
num_class = len(set([label for (label, text) in news]))
vocab_size = len(vocab)
emsize = 64
hidden_dim = 16

## 定义资料转换函数

In [12]:
text_pipeline = lambda x: vocab(tokenizer(x)) # 分词、取得单字的索引值
label_pipeline = lambda x: 0 if x == 'neg' else 1 # 换成索引值

In [13]:
# 测试资料转换
print(text_pipeline('here is an example'))
label_pipeline('pos')

[131, 9, 40, 464]


1

## 建立模型

In [83]:
from torch import nn

class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.rnn = nn.LSTM(embed_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        rnn_out, h_out = self.rnn(embedded)
        return self.fc(rnn_out)

model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

## 定义训练及评估函数

In [84]:
import time

# 训练函数
def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

# 评估函数
def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

## 建立DataLoader，逐批训练

In [85]:
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# 批次处理
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0)) # 设定每笔资料的起始位置
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)  # 单字的索引值累加
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)

train_iter, test_iter = IMDB()
# 转换为 DataSet
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
# 资料切割，95% 作为训练资料
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

# 建立DataLoader
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

## 模型训练

In [86]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

total_accu = None
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time: 16.24s | valid accuracy    0.521 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time: 16.22s | valid accuracy    0.754 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time: 17.14s | valid accuracy    0.784 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time: 16.53s | valid accuracy    0.807 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time: 16.45s | valid accuracy    0.838 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time: 16.38s |

## 模型评估

In [87]:
print(f'测试资料准确度: {evaluate(test_dataloader):.3f}')

测试资料准确度: 0.874


## 测试新资料

In [88]:
# 预测
label = {0:'负面', 1:'正面'}
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text)).to(device)
        output = model(text, torch.tensor([0])).to(device)
        return output.argmax(1).item()

# 测试资料
my_test = open('./nlp_data/imdb_1.txt', encoding='utf8').read()
print(label[predict(data[1], text_pipeline)])

负面


In [89]:
imdb_iterator = iter(IMDB(split='train'))
label_rev = {'neg':0, 'pos':1}
acc = 0
for i in range(20000):
    data = next(imdb_iterator)
    acc += 1 if label_rev[data[0]] == predict(data[1], text_pipeline) else 0
print(acc)

18177
